In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [ ]:
using LinearAlgebra, Combinatorics
using Printf
using Dates

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
#include("load_geometry_data.jl")        # low-level parsers FIRST
#include("GeometryDataLoader.jl")        # defines GeometryDataset
# --- include all modules ---
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")


using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU
using .CriticalPoints: compute_critical_data

In [3]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, -2.7745276335252114, -0.9809436521275706, -1.6990442448471226],
        [0.0, 0.0, 0.0, -3.398088489694245],
        [-0.24028114141347542, -0.6936319083813029, -0.9809436521275706, -1.6990442448471226],
        [0.0, 0.0, -2.942830956382712, -1.6990442448471226]
    ],

    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, -2.7745276335252114, -0.9809436521275706, -1.6990442448471226],
        [0.0, 0.0, 0.0, -3.398088489694245],
        [0.0, 0.0, -2.942830956382712, -1.6990442448471226],
        [0.8981365593438019, 2.7437225604241213, -0.9809436521275707, -1.6990442448471226]
    ],

    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, -3.398088489694245],
        [-0.24028114141347542, -0.6936319083813029, -0.9809436521275706, -1.6990442448471226],
        [0.0, 0.0, -2.942830956382712, -1.6990442448471226],
        [0.8981365593438019, 2.7437225604241213, -0.9809436521275707, -1.6990442448471226]
    ]
]

3-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, -2.7745276335252114, -0.9809436521275706, -1.6990442448471226], [0.0, 0.0, 0.0, -3.398088489694245], [-0.24028114141347542, -0.693631908381303, -0.9809436521275706, -1.6990442448471226], [0.0, 0.0, -2.942830956382712, -1.6990442448471226]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, -2.7745276335252114, -0.9809436521275706, -1.6990442448471226], [0.0, 0.0, 0.0, -3.398088489694245], [0.0, 0.0, -2.942830956382712, -1.6990442448471226], [0.8981365593438019, 2.7437225604241213, -0.9809436521275707, -1.6990442448471226]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -3.398088489694245], [-0.24028114141347542, -0.693631908381303, -0.9809436521275706, -1.6990442448471226], [0.0, 0.0, -2.942830956382712, -1.6990442448471226], [0.8981365593438019, 2.7437225604241213, -0.9809436521275707, -1.6990442448471226]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    bdypoints = bdypoints_all[s]
    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:
Boundary points for simplex 2:
Boundary points for simplex 3:


In [5]:
geom = GeometryCollection(datasets);

four_simplices = [[1,2,3,4,5], [1,2,3,5,6], [1,3,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

[1, 2, 3]


In [6]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [7]:
run_face_xi_matching(geom);

  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [8]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 2.4997123797838998e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 1.2075270435462621e-15).
✓ Closure condition satisfied (max residual = 1.6386370948030818e-15).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------



In [9]:
run_su2_su11_gauge_fix(geom);

In [10]:
include("CriticalPoints.jl")
using .CriticalPoints: compute_critical_data

In [11]:
γ = 1
crit = CriticalPoints.compute_critical_data(geom; gamma = γ)

(gdataof = Vector{Matrix{ComplexF64}}[[[1.0163669568069524 + 0.0im -0.060554650146262254 + 0.1712744150032043im; -0.060554650146262254 - 0.1712744150032043im 1.0163669568069524 + 3.643083215079946e-18im], [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im], [1.1646949512180456 + 0.0im -0.060554650146262136 - 0.08563720750160198im; -0.060554650146262136 + 0.08563720750160198im 0.8680389623958595 - 6.535498186359405e-19im], [1.032474494492746 - 0.0im -2.7755575615628895e-17 + 0.0im; 0.35765965639608477 - 0.0im 0.968546928116901 - 0.0im], [0.8680389623958592 + 8.673617379884035e-19im -0.06055465014626219 - 0.085637207501602im; -0.06055465014626219 + 0.085637207501602im 1.1646949512180451 - 0.0im]], [[1.0 - 9.943095472625259e-21im -4.1882157741605827e-22 + 2.7755166827958315e-17im; 8.561723236707308e-22 + 2.775593058516383e-17im 1.0 + 9.943095472625259e-21im], [1.0358283282766405 - 6.703034942803166e-18im 0.09002488894150976 - 0.2546288377844288im; 0.09002488894150976 + 0.2546288377844288im